# Introdução ao Rasterio

Se você já trabalhou com imagens de satélite ou MDTs no ArcGIS ou QGIS, o Rasterio é como ter essas mesmas ferramentas em Python! É como passar do modo visual para o modo programável de processamento de imagens.

## Objetivos
- Entender dados matriciais (como trabalhar com pixels)
- Aprender a ler e escrever rasters (como importar/exportar imagens)
- Compreender metadados (como ler as informações técnicas)
- Fazer operações básicas (como processamento de imagens)

## 1. Configuração do Ambiente

Vamos preparar nosso laboratório de processamento de imagens digital:

In [ ]:
# Nossa estação de processamento de imagens:
import rasterio           # Nossa ferramenta principal (como o ArcGIS Raster)
import numpy as np        # Nossa calculadora matricial
import matplotlib.pyplot as plt  # Nossa ferramenta de visualização
from rasterio.plot import show   # Para mostrar rasters
from rasterio.transform import from_origin  # Para georreferenciar

# Configurando o tamanho dos nossos mapas
plt.rcParams['figure.figsize'] = (12, 8)

## 2. Conceitos Básicos

### 2.1 O que são Dados Matriciais?

Imagine uma folha quadriculada gigante:
- Cada quadradinho é um pixel (como um sensor de câmera)
- Todos os pixels têm o mesmo tamanho (como uma grade regular)
- Cada pixel guarda um valor (como uma medição)
- Podem ter várias camadas (como RGB numa foto)

Exemplos comuns:
- Imagens de satélite (como fotos aéreas)
- MDTs (como uma maquete digital do terreno)
- Mapas de temperatura (como uma termografia)
- Dados de precipitação (como um radar meteorológico)

## 3. Leitura e Visualização de Dados Raster

In [ ]:
# Vamos criar um raster de exemplo
# Como gerar uma imagem sintética para testes
def criar_raster_exemplo(output_path):
    # Criando dados de exemplo (como um gradiente de cores)
    data = np.linspace(0, 1, 100).reshape(10, 10)
    
    # Definindo o georreferenciamento
    # Como colocar coordenadas na imagem
    transform = from_origin(-46.65, -23.55, 0.01, 0.01)
    
    # Criando o arquivo
    # Como salvar uma imagem georreferenciada
    with rasterio.open(
        output_path,
        'w',
        driver='GTiff',        # Formato (como TIFF)
        height=data.shape[0],  # Altura em pixels
        width=data.shape[1],   # Largura em pixels
        count=1,               # Número de bandas
        dtype=data.dtype,      # Tipo de dado
        crs='EPSG:4326',       # Sistema de coordenadas
        transform=transform    # Georreferenciamento
    ) as dst:
        dst.write(data, 1)    # Salvando os dados

# Criando nosso arquivo de teste
criar_raster_exemplo('../data/processed/exemplo.tif')

# Lendo o raster
# Como abrir uma imagem no SIG
with rasterio.open('../data/processed/exemplo.tif') as src:
    # Lendo os dados
    data = src.read(1)
    
    # Mostrando informações técnicas
    # Como ver as propriedades da imagem
    print("Informações do nosso Raster:")
    print(f"Tamanho: {src.width}x{src.height} pixels")
    print(f"Número de bandas: {src.count}")
    print(f"Sistema de coordenadas: {src.crs}")
    print(f"Transformação: {src.transform}\n")
    
    # Visualizando o raster
    # Como mostrar a imagem na tela
    fig, ax = plt.subplots()
    show(src, ax=ax)
    plt.title('Nosso Raster de Exemplo')
    plt.colorbar(label='Valor')
    plt.show()

## 4. Metadados e Propriedades

In [ ]:
# Explorando os metadados
# Como ler a ficha técnica da imagem
with rasterio.open('../data/processed/exemplo.tif') as src:
    # Extensão geográfica
    # Como ver os limites do mapa
    print("Extensão do raster:")
    print(f"Oeste: {src.bounds.left}")
    print(f"Sul: {src.bounds.bottom}")
    print(f"Leste: {src.bounds.right}")
    print(f"Norte: {src.bounds.top}\n")
    
    # Resolução espacial
    # Como ver o tamanho do pixel no terreno
    print("Resolução:")
    print(f"X: {src.res[0]} graus")
    print(f"Y: {src.res[1]} graus\n")
    
    # Estatísticas básicas
    # Como calcular valores gerais
    data = src.read(1)
    print("Estatísticas:")
    print(f"Mínimo: {data.min()}")
    print(f"Máximo: {data.max()}")
    print(f"Média: {data.mean():.2f}")
    print(f"Desvio padrão: {data.std():.2f}")

## 5. Operações Básicas com Rasters

In [ ]:
# Vamos fazer algumas operações básicas
# Como processar uma imagem
with rasterio.open('../data/processed/exemplo.tif') as src:
    data = src.read(1)
    profile = src.profile
    
    # Operações matemáticas
    # Normalização (como ajuste de contraste)
    data_norm = (data - data.min()) / (data.max() - data.min())
    
    # Classificação (como fatiamento)
    data_class = (data_norm * 5).astype(int)
    
    # Visualização
    fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15, 5))
    
    # Dados originais
    im1 = ax1.imshow(data)
    ax1.set_title('Original')
    plt.colorbar(im1, ax=ax1)
    
    # Dados normalizados
    im2 = ax2.imshow(data_norm)
    ax2.set_title('Normalizado')
    plt.colorbar(im2, ax=ax2)
    
    # Dados classificados
    im3 = ax3.imshow(data_class, cmap='viridis')
    ax3.set_title('Classificado')
    plt.colorbar(im3, ax=ax3)
    
    plt.tight_layout()
    plt.show()
    
    # Salvando o resultado classificado
    # Como exportar uma nova imagem
    profile.update(dtype=rasterio.int32)
    with rasterio.open('../data/processed/exemplo_class.tif', 'w', **profile) as dst:
        dst.write(data_class, 1)

## 6. Máscaras e NoData

In [ ]:
# Trabalhando com máscaras
# Como recortar ou filtrar uma imagem
with rasterio.open('../data/processed/exemplo.tif') as src:
    data = src.read(1)
    profile = src.profile
    
    # Criando uma máscara
    # Como selecionar pixels por valor
    mascara = data > 0.5
    data_mascarado = data.copy()
    data_mascarado[~mascara] = src.nodata if src.nodata else -9999
    
    # Visualização
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
    
    im1 = ax1.imshow(data)
    ax1.set_title('Original')
    plt.colorbar(im1, ax=ax1)
    
    im2 = ax2.imshow(data_mascarado, mask=~mascara)
    ax2.set_title('Com Máscara')
    plt.colorbar(im2, ax=ax2)
    
    plt.tight_layout()
    plt.show()

## 7. Exercícios Práticos

Vamos praticar como um analista de sensoriamento remoto:

1. Manipulação Básica:
   - Crie diferentes padrões de raster
   - Aplique operações matemáticas
   - Visualize os resultados
   - Como processar uma imagem

2. Análise de MDT:
   - Carregue um modelo de elevação
   - Calcule estatísticas por região
   - Crie uma máscara de altitude
   - Como analisar o relevo

3. Transformações:
   - Aplique diferentes classificações
   - Normalize os valores
   - Compare os resultados
   - Como melhorar uma imagem